# 1D Place-Cell Decoding (Toy Example)

This notebook simulates a 1D trajectory and decodes position from Poisson place-cell activity.

In [ ]:
import numpy as np

from tneuro.decoding.bayes_1d import decode_position

rng = np.random.default_rng(0)
dt = 0.05
t = np.arange(0.0, 10.0, dt)
n_time = t.size

pos_min, pos_max = -1.0, 1.0
pos_true = np.sin(2.0 * np.pi * t / t[-1])

n_neurons = 12
centers = np.linspace(pos_min, pos_max, n_neurons)
width = 0.3
peak_rate = 15.0

pos_grid = np.linspace(pos_min, pos_max, 121)

def tuning_rates(x: np.ndarray) -> np.ndarray:
    return peak_rate * np.exp(-0.5 * ((x[None, :] - centers[:, None]) / width) ** 2)

rates = tuning_rates(pos_true)
spike_counts = rng.poisson(rates * dt)

posterior, pos_grid = decode_position(
    spike_counts,
    pos_grid=pos_grid,
    place_centers=centers,
    place_width=width,
    peak_rate_hz=peak_rate,
    dt_s=dt,
)

pos_map = pos_grid[np.argmax(posterior, axis=1)]
rmse = float(np.sqrt(np.mean((pos_map - pos_true) ** 2)))
print(f"MAP position RMSE: {rmse:.3f}")


In [ ]:
try:
    import matplotlib.pyplot as plt
except Exception as exc:
    print("matplotlib not available:", exc)
    plt = None

if plt is not None:
    plt.figure(figsize=(8, 3))
    plt.plot(t, pos_true, label="true", linewidth=2)
    plt.plot(t, pos_map, label="MAP", linewidth=1)
    plt.xlabel("time (s)")
    plt.ylabel("position")
    plt.legend()
    plt.tight_layout()
    plt.show()
